# Import packages

In [ ]:
import pandas as pd
import numpy as np
import mne
from mne_features.feature_extraction import FeatureExtractor
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

# Different classifiers
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Hyperparameters tuning
import optuna

# Functions

In [ ]:
def process_signal(eeg_signal, common_channels, sampling_rate, new_sampling_rate):
    '''
    Preprocess EEG signals by the MNE package.
    
    Parameters include EEG signal, a list of common scalp channel names
    , sampling rate of the original signal, and new targeted sampling rate, respectively.
    '''
    
    # Import data into the MNE package
    eeg_signal = (1e-6) * (eeg_signal) # Convert data from microvolts to volts
    ch_types = ['eeg' for _ in common_channels]
    info = mne.create_info(common_channels, sampling_rate, ch_types = ch_types)
    eeg_signal_mne = mne.io.RawArray(eeg_signal.T, info) # transpose and import the eeg signal into the MNE package
    
    # Apply filtering procedures and downsampling
    eeg_signal_mne.notch_filter(np.asarray([50,100], dtype=float)) # Mitigate powerline interference and retain high freqs
    eeg_signal_mne.filter(1, 125, fir_design = 'firwin') # Band pass filter on 1 and 125 Hz
    eeg_signal_mne.resample(new_sampling_rate, npad = "auto")  # Resampling to the new_sampling_rate (Hz)
    processed_eeg_signal = (eeg_signal_mne.to_data_frame()).drop(labels = "time", axis = 1)
    
    eeg_signal = np.asarray(processed_eeg_signal).T
    
    return eeg_signal



def extract_features(eeg_signal, window_size, window_step, new_sampling_rate):   
    '''
    Extract features from EEG channels by the MNE subpackage (mne_features).
    
    Parameters include processed EEG signal, the size of the window for feature extraction (5 seconds in this paper)
    , the size of the window step (5 seconds in this study to apply non-overlapping)
    , and new targeted sampling rate, respectively.
    '''
    
    dataset = []
    labels = []
    
    for j in range(0, int(eeg_signal.shape[1])-window_size, window_step):
        dataset_array = eeg_signal[: , j : j + window_size] # Select "window_size" from all channels
        dataset_array = np.expand_dims(dataset_array, axis = 1)
        dataset_array = dataset_array.reshape(1,1,window_size * int(eeg_signal.shape[0]))

        # Extract 53 features (some APIs return more than one feature)
        fe = FeatureExtractor(sfreq = new_sampling_rate
                              , selected_funcs = ['mean', 'variance', 'std','ptp_amp', 'skewness'
                                                  , 'kurtosis' ,'rms', 'quantile', 'decorr_time'
                                                  ,'pow_freq_bands', 'hjorth_mobility_spect'
                                                  , 'hjorth_complexity_spect','hjorth_mobility'
                                                  , 'hjorth_complexity', 'higuchi_fd','katz_fd'
                                                  , 'zero_crossings', 'line_length','spect_slope'
                                                  , 'spect_entropy', 'energy_freq_bands', 'spect_edge_freq'
                                                  ,'wavelet_coef_energy', 'teager_kaiser_energy'
                                                  , 'max_cross_corr', 'phase_lock_val', 'nonlin_interdep'])
        # Extra APIs: 'hurst_exp','app_entropy', 'samp_entropy', 'svd_entropy','svd_fisher_info', 'time_corr', 'spect_corr'
            
        dataset_array = fe.fit_transform(dataset_array)
        dataset.extend(dataset_array)

        # Assign labels
        chunk_size = 30 * 60 * new_sampling_rate # 30 minutes
        if j < chunk_size: # Preictal
            labels.append(0) 
        elif j >= chunk_size * 8: # Interictal
            labels.append(1) 

    dataset = np.asarray(dataset)
    labels = np.asarray(labels)

    return dataset, labels

# Fit models

## 1) Randomized Cross-Validation validation method

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True) # 5 folds cross-validation
sc = StandardScaler() # Feature scaling

# Select classifier
classifier = XGBClassifier(eval_metric='mlogloss', use_label_encoder =False) # XGBoost
#classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0) # Random Forest
#classifier = GaussianNB() # Naive Bayes
#classifier = LogisticRegression(max_iter=1000, random_state = 0) # Logistic Regression
#classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # K-Nearest Neighbors (K-NN)
#classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0) # Decision Tree
#classifier = SVC(kernel = 'rbf', random_state = 0) # Kernel SVM
#classifier = SVC(kernel = 'linear', random_state = 0) # Support Vector Machine (SVM)

cm = np.zeros((2, 2))
accuracy = []
sensitivity = []
specificity = []

for i, j in cv.split(dataset):
    X_train, y_train, X_test, y_test = dataset[i], labels[i], dataset[j], labels[j]
        
    # Dataset balancing (apply customized balancing function "balance_dataset")
    X_train, y_train = balance_dataset(X_train, y_train)

    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    cm += confusion_matrix(y_test, y_pred)
    
    accuracy.append(accuracy_score(y_test, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity.append(tp / (tp+fn)) 
    specificity.append(tn / (tn+fp))

# Plot confusion matrix
time_sections = ["preictal", "interictal"]
sns.heatmap(cm, cmap='Blues', annot=True, fmt='g', xticklabels = time_sections
            , yticklabels = time_sections, annot_kws={"size": 14})
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title( "K-fold Method" )
plt.show()

# Print performance metrics values
print("================================================")
print("accuracy: {:.2f} %".format(np.asarray(accuracy).mean()*100))
print("sensitivity: {:.2f} %".format(np.asarray(sensitivity).mean()*100))
print("specificity: {:.2f} %".format(np.asarray(specificity).mean()*100))
print("================================================")

## Tuning hyperparameters (XGBoost) by the Optuna framework

In [ ]:
def objective(trial):
    '''
    Find the best hyperparameters.

    '''
        
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    sc = StandardScaler() # Feature scaling
    accuracies = []

    for i, j in cv.split(dataset): 
        X_train, y_train, X_test, y_test = dataset[i], labels[i], dataset[j], labels[j]

        # Dataset balancing (apply customized balancing function "balance_dataset")
        X_train, y_train = balance_dataset(X_train, y_train)
    
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        param = {
            "eval_metric": "mlogloss",
            "verbosity": 0,
            "objective": "binary:logistic",
            # use exact for small dataset.
            "tree_method": "exact",
            # defines booster, gblinear for linear functions.
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            # L2 regularization weight.
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            # L1 regularization weight.
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            # sampling ratio for training data.
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            # sampling according to each tree.
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        }

        if param["booster"] in ["gbtree", "dart"]:
            # maximum depth of the tree, signifies complexity of the tree.
            param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
            # minimum child weight, larger the term more conservative the tree.
            param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            # defines how selective algorithm is.
            param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
        
        
        #pruning_callback = optuna.integration.XGBoostPruningCallback(trial, observation_key="validation_-auc")
        #classifier = XGBClassifier(**param, fit_params={'callbacks': [pruning_callback]})
        
        classifier = XGBClassifier(**param)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        
    return np.asarray(accuracies).mean()
        
        
        
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("================================================")
trial = study.best_trial
print("Number of finished trials: ", len(study.trials))
print("Best Accuracy: {:.2f} %".format(trial.value*100))
print("Best Hyperparameters: {}".format(trial.params))
print("================================================")

In [ ]:
st = study.trials_dataframe()
print(st)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

## 2) Left One patient Out validation method

### Import one patient's data into the test set and the rest into the training set instead of defining "cv" and applying "KFold". Everything will then continue as before.